In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
data=pd.read_csv(r'C:\Users\Pardhu\OneDrive\Desktop\SMSSpamCollection',sep='\t',header=None)

In [3]:
data.columns=['labels','msgs']
data.head()

,labels,msgs
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [5]:
def main(txt):
    no_punc="".join([a for a in txt if a not in punctuation])
    toke=word_tokenize(no_punc)
    b=[ps.stem(b) for b in toke if b not in stop]
    return b

In [6]:
data['new']=data['msgs'][0:10].apply(main)

In [7]:
data.head()

,labels,msgs,new
0,ham,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazi, avail, bugi, n, gre..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joke, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, earli, hor, u, c, alreadi, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, dont, think, goe, usf, live, around, ..."


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
bag=CountVectorizer(analyzer=main).fit(data['msgs'])

In [9]:
print(bag.vocabulary_)

{'go': 3327, 'jurong': 4127, 'point': 5636, 'crazi': 2234, 'avail': 1321, 'bugi': 1732, 'n': 4958, 'great': 3416, 'world': 7937, 'la': 4274, 'e': 2679, 'buffet': 1730, 'cine': 2014, 'got': 3379, 'amor': 1127, 'wat': 7726, 'ok': 5258, 'lar': 4309, 'joke': 4093, 'wif': 7847, 'u': 7430, 'oni': 5290, 'free': 3139, 'entri': 2792, '2': 414, 'wkli': 7895, 'comp': 2110, 'win': 7860, 'fa': 2912, 'cup': 2281, 'final': 3014, 'tkt': 7233, '21st': 434, 'may': 4670, '2005': 421, 'text': 7083, '87121': 844, 'receiv': 5965, 'questionstd': 5875, 'txt': 7417, 'ratetc': 5922, 'appli': 1201, '08452810075over18': 72, 'dun': 2666, 'say': 6240, 'earli': 2684, 'hor': 3710, 'c': 1775, 'alreadi': 1099, 'nah': 4965, 'i': 3799, 'dont': 2586, 'think': 7159, 'goe': 3339, 'usf': 7549, 'live': 4428, 'around': 1241, 'though': 7176, 'freemsg': 3147, 'hey': 3617, 'darl': 2337, '3': 524, 'week': 7765, 'word': 7926, 'back': 1368, 'id': 3816, 'like': 4397, 'fun': 3204, 'still': 6787, 'tb': 7025, 'xxx': 8056, 'std': 6764, '

In [10]:
print(bag.get_feature_names())

['0', '008704050406', '0089mi', '0121', '01223585236', '01223585334', '0125698789', '02', '020603', '0207', '02070836089', '02072069400', '02073162414', '02085076972', '020903', '021', '050703', '0578', '06', '060505', '061104', '07008009200', '07046744435', '07090201529', '07090298926', '07099833605', '071104', '07123456789', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '0776xxxxxxx', '07786200117', '077xxx', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '0784987', '0789xxxxxxx', '0794674629107880867867', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986030', '08002986906', '08002988890', '08006344447', '0808', '08081263000', '08081560665', '0825', '0844', '08448350055', '08448714184', '0845', '08450542832', '08452810071', '08452810073', '08452810075over18', '0870', '08700621170150p', '08701213186', 

In [11]:
vect_bag=bag.transform(data['msgs'])

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer


In [13]:
norm=TfidfTransformer().fit(vect_bag)

In [14]:
new_norm=norm.transform(vect_bag)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(new_norm,data['labels'],test_size=0.2)

In [17]:
from sklearn.naive_bayes import MultinomialNB


In [18]:
mb=MultinomialNB()

In [19]:
mb.fit(x_train,y_train)

MultinomialNB()

In [20]:
y_res=mb.predict(x_test)

In [21]:
from sklearn.metrics import confusion_matrix,classification_report

In [22]:
print(confusion_matrix(y_test,y_res))

[[962   0]
 [ 44 109]]


In [23]:
print(classification_report(y_test,y_res))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       962
        spam       1.00      0.71      0.83       153

    accuracy                           0.96      1115
   macro avg       0.98      0.86      0.90      1115
weighted avg       0.96      0.96      0.96      1115



In [24]:
msg_train,msg_test,label_train,label_test=train_test_split(data['msgs'],data['labels'],test_size=0.2)

In [25]:
from sklearn.pipeline import Pipeline

In [26]:
pipe=Pipeline([('cv',CountVectorizer(analyzer=main)),('tf',TfidfTransformer()),('mb',MultinomialNB())])

In [27]:
pipe.fit(msg_train,label_train)

Pipeline(steps=[('cv',
                 CountVectorizer(analyzer=<function main at 0x0000022EDE0A83A0>)),
                ('tf', TfidfTransformer()), ('mb', MultinomialNB())])

In [28]:
res=pipe.predict(msg_test)

In [29]:
print(classification_report(label_test,res))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.98       956
        spam       1.00      0.70      0.82       159

    accuracy                           0.96      1115
   macro avg       0.98      0.85      0.90      1115
weighted avg       0.96      0.96      0.95      1115

